<a href="https://colab.research.google.com/github/KMadhu10/AIML_1374/blob/main/2303A51374_B06.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Prediction of Opportunities in Turkish

Q1. Identify the Top-10 Startups in Turkey


Q2. Find the Max and Min crowdsourced funding obtained by Turkey

Q3.Name the gender which has acquired the most funding for Startups

Q4.Identify the Top-5 cities where startup is most

Q5.Name the Top-5 technologies popular in Turkish startups.

Q6. Apply either Classification or Clustering or Regression Model and evaluate the accuracy, error metrics of the model.




Q1. Identify the Top-10 Startups in Turkey

In [27]:
import pandas as pd


data = {
    'Startup': ['Getir', 'Trendyol', 'Peak Games', 'Insider', 'Hepsiburada', 'Yemeksepeti', 'Letgo', 'Armut', 'Mutlubiev', 'Paraşüt'],
        'Valuation': [1000, 800, 600, 500, 400, 300, 200, 150, 120, 100]  # Example valuation or funding data
        }

df = pd.DataFrame(data)


top_10_startups = df.sort_values(by=['Valuation'], ascending=False).head(10)


print(top_10_startups)

       Startup  Valuation
0        Getir       1000
1     Trendyol        800
2   Peak Games        600
3      Insider        500
4  Hepsiburada        400
5  Yemeksepeti        300
6        Letgo        200
7        Armut        150
8    Mutlubiev        120
9      Paraşüt        100


Q2. Find the Max and Min crowdsourced funding obtained by Turkey

In [28]:
# Assuming your dataset is in a CSV file named 'turkish_startups.csv'
df = pd.read_csv('turkish_crowdfunding_startups.csv')

In [29]:
import pandas as pd

df = pd.read_csv('turkish_crowdfunding_startups.csv')

# Print all column names
print(df.columns)

# Display the first few rows to inspect the data
print(df.head())

Index(['id;platform_adi;kitle_fonlamasi_turu;kategori;fon_sekli;proje_adi;proje_sahibi;proje_sahibi_cinsiyet;kac_proje_destekledi;kac_projeye_abone;kac_projenin_sahibi;kac_proje_takiminda;konum;bolge;yil;proje_baslama_tarihi;proje_bitis_tarihi;gun_sayisi;tanitim_videosu;video_uzunlugu;gorsel_sayisi;sss;guncellemeler;yorumlar;destekci_sayisi;odul_sayisi;ekip_kisi_sayisi;web_sitesi;sosyal_medya;sm_sayisi;sm_takipci;etiket_sayisi;icerik_kelime_sayisi;proje_aciklamasi;hedef_miktari;toplanan_tutar;destek_orani;basari_durumu',
       'Unnamed: 1', 'Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4', 'Unnamed: 5',
       'Unnamed: 6', 'Unnamed: 7', 'Unnamed: 8'],
      dtype='object')
  id;platform_adi;kitle_fonlamasi_turu;kategori;fon_sekli;proje_adi;proje_sahibi;proje_sahibi_cinsiyet;kac_proje_destekledi;kac_projeye_abone;kac_projenin_sahibi;kac_proje_takiminda;konum;bolge;yil;proje_baslama_tarihi;proje_bitis_tarihi;gun_sayisi;tanitim_videosu;video_uzunlugu;gorsel_sayisi;sss;guncellemeler;yorumlar;des

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.preprocessing import LabelEncoder
from sentence_transformers import SentenceTransformer
from catboost import CatBoostClassifier

import optuna
import shap
import warnings
warnings.filterwarnings('ignore')

In [ ]:
df = pd.read_csv('/kaggle/input/turkish-crowdfunding-startups/turkishCF.csv', sep=';', on_bad_lines='skip')

In [42]:
!pip install sentence_transformers

In [45]:
pd.set_option('display.max_columns', None)

In [47]:
df.head()

,id,platform_adi,kitle_fonlamasi_turu,kategori,fon_sekli,proje_adi,proje_sahibi,proje_sahibi_cinsiyet,kac_proje_destekledi,kac_projeye_abone,kac_projenin_sahibi,kac_proje_takiminda,konum,bolge,yil,proje_baslama_tarihi,proje_bitis_tarihi,gun_sayisi,tanitim_videosu,video_uzunlugu,gorsel_sayisi,sss,guncellemeler,yorumlar,destekci_sayisi,odul_sayisi,ekip_kisi_sayisi,web_sitesi,sosyal_medya,sm_sayisi,sm_takipci,etiket_sayisi,icerik_kelime_sayisi,proje_aciklamasi,hedef_miktari,toplanan_tutar,destek_orani,"basari_durumu,,,,,,,,"
0,1,fongogo,ödül,diğer,ya hep ya hiç,Gerçek Gizlidir Filmleri,Lob Ekibi,belirsiz,1,0.0,1,0,istanbul,marmara,2014.0,15.09.2014,16.11.2014,62.0,var,104.0,1.0,0.0,4,0,150.0,11.0,4.0,yok,var,3.0,274.0,0,301,Türkiye’de ilk kez her aşaması ‘online’ olarak...,40000.0,54410.0,136%,"başarılı,,,,,,,,"
1,2,fongogo,ödül,diğer,ya hep ya hiç,Fongogo - Hayat Bulsun!,Fongogo Team,belirsiz,3,0.0,2,0,istanbul,marmara,2015.0,4.05.2015,3.07.2015,60.0,yok,0.0,12.0,0.0,0,0,51.0,11.0,7.0,yok,var,3.0,5634.0,0,167,"Fongogo ekibi olarak büyümek, daha geniş kitle...",50000.0,50110.0,100%,"başarılı,,,,,,,"
2,3,fongogo,ödül,diğer,ya hep ya hiç,"PAYLAŞIMIN, SEVGİNİN VE ÇOCUKLARIN İYİLİK EVİ ...",SİNEM CAN,kadın,0,0.0,2,1,belirsiz,belirsiz,2017.0,NaN,13.06.2017,60.0,var,60.0,8.0,1.0,1,1,25.0,6.0,2.0,var,var,2.0,510.0,4,413,Çocuklara her şeyin ücretsiz olduğu iyilik evi...,47000.0,47850.0,101%,"başarılı,,,,,"
3,4,fongogo,ödül,diğer,ya hep ya hiç,Türk Kadının 100.yıl Seyri,Deniz Tutkusu Seyirde,kadın,0,0.0,1,0,muğla,ege,2019.0,19.04.2019,18.06.2019,60.0,var,67.0,6.0,0.0,0,0,45.0,7.0,1.0,var,var,2.0,3338.0,5,287,‘Sözde değil Özde İzindeyiz ‘ sloganıyla Atatü...,35000.0,35450.0,101%,"başarılı,,,,,,,,"
4,5;fongogo;ödül;diğer;ya hep ya hiç;Doğadaki Ço...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
df['promotion_video'] = df['promotion_video'].map(var_yok_rename)
df['website'] = df['website'].map(var_yok_rename)
df['social_media'] = df['social_media'].map(var_yok_rename)
df['success_status'] = df['success_status'].map(success_status_rename)
df['category'] = df['category'].map(category_rename)
df['crowdfunding_type'] = df['crowdfunding_type'].map(kitle_fonlamasi_turu_rename)
df['fund_type'] = df['fund_type'].map(fon_sekli_rename)
df['project_owner_gender'] = df['project_owner_gender'].map(proje_sahibi_rename)
df['region'] = df['region'].map(bolge_rename)

In [ ]:
df.head()

In [ ]:
df.drop(columns=['id','project_owner'], inplace=True)

In [ ]:
plt.figure(figsize=(12, 6))
sns.barplot(x='crowdfunding_type', y='target_amount', data=df, estimator='mean', ci=None,color='cyan', alpha=0.5, label='Target Amount')
sns.barplot(x='crowdfunding_type', y='amount_collected', data=df, estimator='mean', ci=None, color='blue', alpha=0.5, label='Collected Amount')
plt.title('Average Target Amount and Collected Amount by Crowdfunding Type', fontsize=14)
plt.xlabel('Crowdfunding Type')
plt.ylabel('Amount')
plt.legend()
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()